# Taxi Travel Data Analysis

In this demo, we will be doing some demos on temporal feature engineering with the Kaggle Dataset

### Loading libraries, datasets

In [1]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# These are all of the files you are given
df_tr = pd.read_csv("train.csv")
df_public_test = pd.read_csv("test_public.csv")

In [3]:
print(len(df_tr))
print(len(df_public_test))

1710670
320


In [4]:
print(len(df_tr.columns))

9


In [ ]:
df_tr.head()

In [ ]:
def is_holiday(timestamp):
    

In [ ]:
print(f"Training set has {df_tr['MISSING_DATA'].sum()} incomplete datapoints")
print(f"Public test set has {df_public_test['MISSING_DATA'].sum()} incomplete datapoints")

print("train CALL_TYPE == A:", (df_tr["CALL_TYPE"] == "A").sum() / len(df_tr))
print("train CALL_TYPE == B:", (df_tr["CALL_TYPE"] == "B").sum() / len(df_tr))
print("train CALL_TYPE == C:", (df_tr["CALL_TYPE"] == "C").sum() / len(df_tr))

print("public CALL_TYPE == A:", (df_public_test["CALL_TYPE"] == "A").sum() / len(df_public_test))
print("public CALL_TYPE == B:", (df_public_test["CALL_TYPE"] == "B").sum() / len(df_public_test))
print("public CALL_TYPE == C:", (df_public_test["CALL_TYPE"] == "C").sum() / len(df_public_test))


In [ ]:
print("train set:")
for col in ["TRIP_ID","CALL_TYPE","ORIGIN_CALL","ORIGIN_STAND","TAXI_ID","TIMESTAMP","DAY_TYPE","MISSING_DATA"]:
    print(f'{col} has {df_tr[col].isnull().sum()} null rows')

print("\npublic test set:")    
for col in ["TRIP_ID","CALL_TYPE","ORIGIN_CALL","ORIGIN_STAND","TAXI_ID","TIMESTAMP","DAY_TYPE","MISSING_DATA"]:
    print(f'{col} has {df_public_test[col].isnull().sum()} null rows')
print()
temp = df_tr[df_tr["CALL_TYPE"] == "A"]
print(f"CALL_TYPE A has {temp['ORIGIN_CALL'].isnull().sum()} missing ORIGIN_CALLs")
temp = df_tr[df_tr["CALL_TYPE"] == "B"]
print(f"CALL_TYPE B has {temp['ORIGIN_STAND'].isnull().sum()} missing ORIGIN_STANDs")
temp = df_public_test[df_public_test["CALL_TYPE"] == "A"]
print(f"In public test, CALL_TYPE A has {temp['ORIGIN_CALL'].isnull().sum()} missing ORIGIN_CALLs")
temp = df_public_test[df_public_test["CALL_TYPE"] == "B"]
print(f"In public test, CALL_TYPE B has {temp['ORIGIN_STAND'].isnull().sum()} missing ORIGIN_STANDs")


In [ ]:
df_public_test.head()

### Get Computed Time from POLYLINE

Our goal is to predict the travel-time of the taxi, which can be derived from the POLYLINE length.

Recall:

```
The travel time of the trip (the prediction target of this project) is defined as the (number of points-1) x 15 seconds. 
For example, a trip with 101 data points in POLYLINE has a length of (101-1) * 15 = 1500 seconds. Some trips have missing 
data points in POLYLINE, indicated by MISSING_DATA column, and it is part of the challenge how you utilize this knowledge.
```

We are not doing anything with the MISSING_DATA. It is up to you to find a way to use (or ignore) that information.

In [ ]:
# Over every single 
def polyline_to_trip_duration(polyline):
  return max(polyline.count("[") - 2, 0) * 15

# This code creates a new column, "LEN", in our dataframe. The value is
# the (polyline_length - 1) * 15, where polyline_length = count("[") - 1
df_tr["LEN"] = df_tr["POLYLINE"].apply(polyline_to_trip_duration)

In [ ]:
from datetime import datetime
def parse_time(x):
  # We are using python's builtin datetime library
  # https://docs.python.org/3/library/datetime.html#datetime.date.fromtimestamp

  # Each x is essentially a 1 row, 1 column pandas Series
  dt = datetime.fromtimestamp(x["TIMESTAMP"])
  return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

# Because we are assigning multiple values at a time, we need to "expand" our computed (year, month, day, hour, weekday) tuples on 
# the column axis, or axis 1
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html
df_tr[["YR", "MON", "DAY", "HR", "WK"]] = df_tr[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")

In [ ]:
def parse_timestamp(x):
  # We are using python's builtin datetime library
  # https://docs.python.org/3/library/datetime.html#datetime.date.fromtimestamp

  # Each x is essentially a 1 row, 1 column pandas Series
  dt = datetime.fromtimestamp(x)
  return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

def is_holiday(timestamp):
    yr, mon, day, _hr, _wk = parse_timestamp(timestamp)
#     print(f"year {yr}, month {mon}, day {day}, hour {_hr}, week {_wk}")
    holidays = [
        (2013, 1, 1),
        (2014, 1, 1),
        (2013, 2, 12),
        (2014, 3, 4),
        (2013, 3, 29),
        (2014, 4, 18),
        (2013, 3, 31),
        (2014, 4, 20),
        (2013, 4, 25),
        (2014, 4, 25),
        (2013, 5, 1),
        (2014, 5, 1),
        (2013, 6, 10),
        (2014, 6, 10),
        (2013, 8, 15),
        (2014, 8, 15),
        (2013, 10, 5),
        (2014, 10, 5),
        (2013, 11, 1),
        (2014, 11, 1),
        (2013, 12, 1),
        (2014, 12, 1),
        (2013, 12, 25),
        (2014, 12, 25)
    ]
    if (yr, mon, day) in holidays:
        return True
    return False

def is_holidays_eve(timestamp):
    tomorrow = timestamp + (60 * 60 * 24)
    return is_holiday(tomorrow)



In [ ]:
print(parse_time())

### Do some Feature Analysis

For our feature analysis, we are looking at which of our engineered features may be useful in making a taxicab time regression model

### Create a Prediction File

In [ ]:
mean, std = df_tr["LEN"].mean(), df_tr["LEN"].std()
median = df_tr["LEN"].median()
print(f"{mean=} {median=} {std=}")

In [ ]:
# Sample submission file that is given on kaggle
df_sample = pd.read_csv("sampleSubmission.csv")

df_sample["TRAVEL_TIME"] = 716.43

# mean(716.43) -> 792.73593
# median(600) -> 784.74219
# df_sample.to_csv("my_pred.csv", index=None)

In [ ]:
# First n samples to analyze. Set to -1 to use all data
end = -1

outlier_threshold = 3

# "Choose all data, where the trip length is less than 3 standard deviations away from the mean"
# This is to remove outliers. Otherwise, our plots would look very squished (since there are some
# VERRRRRY long taxi trips in the dataset)
df_trimmed = df_tr[df_tr["LEN"] < mean + outlier_threshold * std]

# Because our y-values only take on multiples of 15, we want just enough buckets in a histogram
# such that each buckets counts one value's frequency. (e.x. one bucket counts how many 15s trips, 
# how many 30s trips, etc. )
buckets = (int(mean + outlier_threshold * std) // 15)

print(f"Using: {len(df_trimmed)}/{len(df_tr)}")

fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(18,14))

# Now, we visualize some features that we think might be useful
for idx, v in enumerate(["YR", "MON", "DAY", "HR", "WK", "ORIGIN_STAND", "ORIGIN_CALL",
                        "TAXI_ID"]):
  # idx // 3 = row, idx % 3 = column
  ax = axs[idx // 3, idx % 3]
  
  # Remove any rows with invalid values
  df_subset = df_trimmed.dropna(subset=v)
  
  # Create a histogram. Look up the documentation for more details
  ax.hist2d(df_subset[v][:end], df_subset["LEN"][:end], cmap="CMRmap", bins=(120,buckets))
  
  # Some stylistic things to make the graphs look nice
  ax.set_xlim(ax.get_xlim()[0] - 1, ax.get_xlim()[1] + 1)
  ax.set_facecolor("black")
  ax.set_ylabel("seconds", fontsize=18)
  ax.set_title(f"Feature: {v}", fontsize=20)


In [ ]:
plt.figure(figsize=(10,10))
for v in [0, 5, 11, 17, 23]:
  # Filter data where the HR matches v
  hourly_data = df_trimmed[df_trimmed["HR"] == v]["LEN"]
  histogram, bin_boundary = np.histogram(hourly_data, bins=buckets)
  histogram = histogram / len(hourly_data)
  # The center is the left_bound and right_bound of a bucket
  bin_centers = [(bin_boundary[i] + bin_boundary[i + 1]) / 2 for i in range(buckets)]
  plt.plot(bin_centers, histogram, label=f"HR={v}")
plt.legend();

In [ ]:
# def bar_chart(column="MON"):
#     plt.figure(figsize=(4,4))
#     months = sorted(df_tr[column].unique())
#     month_data = []
#     for v in months:
#         month_data.append(df_trimmed[df_trimmed[column] == v]["LEN"].mean())
#     plt.bar(months, month_data)
#     plt.xlabel(column)
#     plt.ylabel("LEN")

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(18,14))


for idx, column in enumerate(["YR", "MON", "DAY", "WK"]):
    # idx // 3 = row, idx % 3 = column
    print(idx)
    ax = axs[idx // 2, idx % 2]
    
#     ax.figure(figsize=(4,4))
    months = sorted(df_tr[column].unique())
    month_data = []
    for v in months:
        month_data.append(df_trimmed[df_trimmed[column] == v]["LEN"].mean())
    ax.bar(months, month_data)
    ax.set_xlabel(column)
    ax.set_ylabel("LEN")
    ax.set_title(column)
    
# ax.set_xlim(ax.get_xlim()[0] - 1, ax.get_xlim()[1] + 1)
#   ax.set_facecolor("black")
#   ax.set_ylabel("seconds", fontsize=18)
#   ax.set_title(f"Feature: {v}", fontsize=20)
  
  # Remove any rows with invalid values
#   df_subset = df_trimmed.dropna(subset=v)
# bar_chart("MON")
# bar_chart("DAY")
# bar_chart("WK")
# bar_chart("YR")





In [ ]:
tr_start_date = pd.to_datetime(df_tr["TIMESTAMP"], unit='s').min()
tr_end_date = pd.to_datetime(df_tr["TIMESTAMP"], unit='s').max()
tt_start_date = pd.to_datetime(df_public_test["TIMESTAMP"], unit='s').min()
tt_end_date = pd.to_datetime(df_public_test["TIMESTAMP"], unit='s').max()
print(f"The training data have dates ranging from {tr_start_date} to {tr_end_date}.")
print(f"The public test data have dates ranging from {tt_start_date} to {tt_end_date}.")

In [ ]:
plt.hist(df_trimmed['LEN'], bins=200)
plt.xlabel('LEN')
plt.ylabel('frequency')
plt.title('training data distribution of LEN (outliers removed)')
plt.show()


In [ ]:
# to get the actual coordinates, not just LEN
def parse_polyline(polyline):
    assert not " " in polyline, "polyline has whitespace!"
    polyline = polyline.strip("[]")
    coords = polyline.split("],[")
    for i in range(len(coords)):
        coords[i] = coords[i].split(",")
        if len(coords[i]) < 2:
            return []
        coords[i][0] = float(coords[i][0])
        coords[i][1] = float(coords[i][1])
    return coords

def get_start(coords):
    if len(coords) == 0:
        return None, None
    return coords[0][0], coords[0][1]

def get_end(coords):
    if len(coords) == 0:
        return None, None
    return coords[-1][0], coords[-1][1]

def get_start_lat(polyline):
    parsed = parse_polyline(polyline)
    return get_start(parsed)[0]

def get_start_lon(polyline):
    parsed = parse_polyline(polyline)
    return get_start(parsed)[1]    

x_coords = []
y_coords = []
for index, row in df_trimmed.iterrows():
    if index > 1000:
        break
    parsed = parse_polyline(row["POLYLINE"])
    for coord in parsed:
        if coord == None or len(coord) != 2 or coord[0] == None or coord[1] == None:
            continue
        x_coords.append(coord[0])
        y_coords.append(coord[1])

In [ ]:

# df_tr["START_LAT"] = df_tr["POLYLINE"].apply(get_start_lat)
# df_tr["START_LON"] = df_tr["POLYLINE"].apply(get_start_lon)

In [ ]:
grid_size = 30

min_y, max_y = min(y_coords), max(y_coords)
min_x, max_x = min(x_coords), max(x_coords)
# min_y, max_y = min(min(y_coords), df_tr["START_LAT"].min()), max(max(y_coords), df_tr["START_LAT"].max())
# min_x, max_x = min(min(x_coords), df_tr["START_LON"].min()), max(max(x_coords), df_tr["START_LON"].max())


inc_y = (max_y - min_y) / grid_size
inc_x = (max_x - min_x) / grid_size


grid = np.zeros((grid_size, grid_size))

for x, y in zip(x_coords, y_coords):
    x_index = int((x - min_x) // (inc_x + 0.0001))
    y_index = int((y - min_y) // (inc_y + 0.0001))
    
    if x_index >= grid_size or y_index >= grid_size:
#         print(x_index, y_index)
        continue

    grid[x_index][y_index] += 1

In [ ]:
# because i can't figure out a cleaner way to set the labels...
def remove_duplicates(labels):
    seen_labels = []
    for i, label in enumerate(labels):
        if label in seen_labels:
            labels[i] = ""
        else:
            seen_labels.append(label)
    return labels
    

In [ ]:
##############################
## Heatmap ###################
##############################

x_labels = remove_duplicates(["{:.1f}".format((inc_x * i) + min_x) for i in range(0, grid_size)])
y_labels = remove_duplicates(["{:.1f}".format((inc_y * i) + min_y) for i in range(0, grid_size)])


sns.heatmap(grid, cbar=False, xticklabels=x_labels, yticklabels=y_labels)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Heatmap of Trip Positions')


plt.show()


